In [1]:
# pip install pandas geopandas shapely osmnx networkx folium

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

boundaryPath = '/Users/mihiarc/Work/data/spatial-boundaries/'

# Load the CSV file
df = pd.read_csv('data/FIA_2023_PLOT_US_MOG.csv')


In [3]:
# Create a geometry column from latitude and longitude
geometry = [Point(xy) for xy in zip(df['LON'], df['LAT'])]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')  # WGS84 coordinate system

In [4]:
# Load US states shapefile
states = gpd.read_file(boundaryPath + 'tl_2023_us_state/tl_2023_us_state.shp')

# List of the 13 southern US states
southern_states = [
    'Alabama', 'Arkansas', 'Florida', 'Georgia', 'Kentucky', 'Louisiana',
    'Mississippi', 'North Carolina', 'South Carolina', 'Tennessee', 'Texas',
    'Virginia', 'West Virginia'
]

# Filter the states GeoDataFrame
southern_states_gdf = states[states['NAME'].isin(southern_states)]

In [5]:
# Ensure both GeoDataFrames use the same coordinate reference system (CRS)
gdf = gdf.to_crs(southern_states_gdf.crs)

# Spatial join to select points within southern states
forests_in_south = gpd.sjoin(gdf, southern_states_gdf, how='inner', predicate='within')

In [6]:
# Load MSA shapefile
msa = gpd.read_file(boundaryPath + 'tl_2020_us_metdiv/tl_2020_us_metdiv.shp')

# Create a GeoDataFrame of MSA centroids
msa_centroids = msa.copy()
msa_centroids['geometry'] = msa.geometry.centroid

/var/folders/c9/dwfk30c575n0dw3tkq6prd940000gn/T/ipykernel_73922/3330247313.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  msa_centroids['geometry'] = msa.geometry.centroid


In [7]:
print(f"Number of forest points: {len(forests_in_south)}")
print(f"Number of MSA centroids: {len(msa_centroids)}")

print(f"Forest geometry types:\n{forests_in_south.geometry.geom_type.unique()}")
print(f"MSA centroid geometry types:\n{msa_centroids.geometry.geom_type.unique()}")

print(f"Any missing forest coordinates: {forests_in_south.geometry.hasnans}")
print(f"Any missing MSA coordinates: {msa_centroids.geometry.hasnans}")

Number of forest points: 47321
Number of MSA centroids: 31
Forest geometry types:
['Point']
MSA centroid geometry types:
['Point']
Any missing forest coordinates: False
Any missing MSA coordinates: False


In [8]:
# Ensure both GeoDataFrames are in the same projected CRS
forests_in_south = forests_in_south.to_crs('EPSG:3857')
msa_centroids = msa_centroids.to_crs('EPSG:3857')

# Ensure both GeoDataFrames are in the same projected CRS
forests_in_south = forests_in_south.to_crs('EPSG:3857')
msa_centroids = msa_centroids.to_crs('EPSG:3857')

# Drop 'index_left' and 'index_right' if they exist
if 'index_left' in forests_in_south.columns:
    forests_in_south = forests_in_south.drop(columns=['index_left'])
if 'index_right' in forests_in_south.columns:
    forests_in_south = forests_in_south.drop(columns=['index_right'])
if 'index_left' in msa_centroids.columns:
    msa_centroids = msa_centroids.drop(columns=['index_left'])
if 'index_right' in msa_centroids.columns:
    msa_centroids = msa_centroids.drop(columns=['index_right'])

# Perform the spatial join
nearest_msa = forests_in_south.sjoin_nearest(
    msa_centroids,
    how='left',
    distance_col='distance'
)

In [9]:
nearest_msa.columns

Index(['INVYR', 'STATECD', 'UNITCD', 'COUNTYCD', 'PLOT', 'PLOT_STATUS_CD',
       'MEASYEAR', 'MEASMON', 'MEASDAY', 'REMPER',
       ...
       'GEOID_right', 'NAME_right', 'NAMELSAD', 'LSAD_right', 'MTFCC_right',
       'ALAND_right', 'AWATER_right', 'INTPTLAT_right', 'INTPTLON_right',
       'distance'],
      dtype='object', length=103)

In [10]:


# nearest_msa = nearest_msa.set_geometry('geometry_left')

nearest_msa['straight_line_distance_km'] = nearest_msa['distance'] / 1000

In [11]:
import osmnx as ox

# Define the bounding box covering the southern states
minx, miny, maxx, maxy = southern_states_gdf.total_bounds

# Download the road network
G = ox.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive')

/var/folders/c9/dwfk30c575n0dw3tkq6prd940000gn/T/ipykernel_73922/2734199762.py:7: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  G = ox.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive')
/var/folders/c9/dwfk30c575n0dw3tkq6prd940000gn/T/ipykernel_73922/2734199762.py:7: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = ox.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive')
/opt/anaconda3/envs/geo/lib/python3.9/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 2,156 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_p

KeyboardInterrupt: 

In [ ]:
# Function to get the nearest network node
def get_nearest_node(row):
    point = row['forest_geometry']
    return ox.distance.nearest_nodes(G, point.x, point.y)

def get_nearest_msa_node(row):
    point = row['msa_centroid_geometry']
    return ox.distance.nearest_nodes(G, point.x, point.y)

# Apply to the DataFrame
nearest_msa['forest_node'] = nearest_msa.apply(get_nearest_node, axis=1)
nearest_msa['msa_node'] = nearest_msa.apply(get_nearest_msa_node, axis=1)

In [ ]:
import networkx as nx

def calculate_route_length(row):
    try:
        length = nx.shortest_path_length(
            G, row['forest_node'], row['msa_node'], weight='length'
        )
        return length  # length in meters
    except nx.NetworkXNoPath:
        return None

nearest_msa['road_distance_m'] = nearest_msa.apply(calculate_route_length, axis=1)
nearest_msa['road_distance_km'] = nearest_msa['road_distance_m'] / 1000

In [ ]:
nearest_msa['travel_time_min'] = (nearest_msa['road_distance_km'] / 60) * 60  # in minutes

In [ ]:
output_table = nearest_msa[[
    'forest_id', 'msa_id', 'straight_line_distance_km',
    'road_distance_km', 'travel_time_min'
]]

# Save to CSV
output_table.to_csv('forest_msa_distances.csv', index=False)

In [ ]:
import folium

# Initialize the map
m = folium.Map(location=[
    forests_in_south.geometry.y.mean(),
    forests_in_south.geometry.x.mean()
], zoom_start=6)

# Add forest points
for _, row in nearest_msa.iterrows():
    folium.CircleMarker(
        location=[row['forest_geometry'].y, row['forest_geometry'].x],
        radius=3,
        color='green',
        fill=True
    ).add_to(m)

# Add MSA centroids
for _, row in nearest_msa.iterrows():
    folium.Marker(
        location=[row['msa_centroid_geometry'].y, row['msa_centroid_geometry'].x],
        popup=row['msa_name']  # Replace with the appropriate column name
    ).add_to(m)

# Draw lines between forests and their nearest MSAs
for _, row in nearest_msa.iterrows():
    folium.PolyLine(
        locations=[
            (row['forest_geometry'].y, row['forest_geometry'].x),
            (row['msa_centroid_geometry'].y, row['msa_centroid_geometry'].x)
        ],
        color='blue'
    ).add_to(m)

# Save the map
m.save('forest_msa_map.html')